# Hands-on session: Machine Learning & Data Visualization
---
## 4 - Best practices in ML pipelines
---


How do I run this?

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sina-mansour/OHBM-Brainhack-2021)

Gitpod?

Binder?

---

## Standardization

---
## Confound regression

---
## Cross-validation

---
## Hyperparemeter tuning

---
## Evaluating overfitting vs. underfitting